In [47]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [48]:
daily = pd.read_csv("./daily_household.csv")
avg_power_kw = daily['Global_active_power'] / 24
daily["load_factor"] = avg_power_kw / daily["peak_hour_power"]
daily.head()

,Datetime,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,peak_hour_power,night_time_usage,...,Day_Sunday.1,Day_Thursday.1,Day_Tuesday.1,Day_Wednesday.1,Season_Autumn,Season_Spring,Season_Summer,Season_Winter,is_working_Day,load_factor
0,2006-12-16,20.152933,34.922,236.243763,13.082828,0.000,0.546,4.926,7.706,1017.334,...,False,False,False,False,False,False,False,True,0,0.108968
1,2006-12-17,56.507667,226.006,240.087028,9.999028,2.033,4.187,13.341,7.064,267.578,...,True,False,False,False,False,False,False,True,0,0.333308
2,2006-12-18,36.730433,161.792,241.231694,6.421667,1.063,2.621,14.018,6.158,344.776,...,False,False,False,False,False,False,False,True,1,0.248528
3,2006-12-19,27.769900,150.942,241.999313,4.926389,0.839,7.602,6.197,7.840,208.098,...,False,False,True,False,False,False,False,True,1,0.147587
4,2006-12-20,37.095800,160.998,242.308063,6.467361,0.000,2.648,14.063,5.988,629.370,...,False,False,False,True,False,False,False,True,1,0.258126


In [49]:
daily["Power_level"] = pd.qcut(daily["Global_active_power"], q=2, labels=["Power_low", "Power_high"])
daily["Weekend"] = (daily["Day_Saturday"] | daily["Day_Sunday"])

df = pd.concat([pd.get_dummies(daily["Power_level"]),
                daily[["Weekend", "Season_Summer", "Season_Spring", "Season_Winter", "Season_Autumn", "is_working_Day"]]], axis=1)

df = df.astype(bool)

In [52]:
freq = apriori(df, min_support=0.1, use_colnames=True)

rules = association_rules(freq, metric="confidence", min_threshold=0.6)
rules = rules.sort_values("lift", ascending=False)

rules.head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
9,"(is_working_Day, Season_Summer)",(Power_low),0.182386,0.500000,0.170596,0.935361,1.870722,1.0,0.079404,7.735294,0.569275,0.333333,0.870722,0.638277
0,(Season_Summer),(Power_low),0.255201,0.500000,0.230236,0.902174,1.804348,1.0,0.102635,5.111111,0.598528,0.438573,0.804348,0.681323
11,(Season_Summer),"(is_working_Day, Power_low)",0.255201,0.391817,0.170596,0.668478,1.706098,1.0,0.070604,1.834519,0.555677,0.358079,0.454898,0.551938
3,(Season_Winter),(Power_high),0.239945,0.500000,0.203883,0.849711,1.699422,1.0,0.083911,3.326923,0.541493,0.380336,0.699422,0.628739
12,"(Season_Winter, is_working_Day)",(Power_high),0.171290,0.500000,0.141470,0.825911,1.651822,1.0,0.055825,2.872093,0.476171,0.267016,0.651822,0.554426
